In [78]:
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
df = pd.read_csv('/content/drive/MyDrive/shapeaiproj/spam.csv' , encoding='ISO-8859-1')
le = LabelEncoder()

In [18]:
data = df.to_numpy()

In [19]:
X = data[:, 1]
y = data[:, 0]

In [20]:
X.shape, y.shape

((5572,), (5572,))

In [21]:
tokenizer = RegexpTokenizer('\w+')
sw = set(stopwords.words('english'))
ps = PorterStemmer()

In [22]:
def getStem(review):
    review = review.lower()
    tokens = tokenizer.tokenize(review) # breaking into small words
    removed_stopwords = [w for w in tokens if w not in sw]
    stemmed_words = [ps.stem(token) for token in removed_stopwords]
    clean_review = ' '.join(stemmed_words)
    return clean_review

In [23]:
# get a clean document
def getDoc(document):
    d = []
    for doc in document:
        d.append(getStem(doc))
    return d

In [24]:
stemmed_doc = getDoc(X)

In [25]:
stemmed_doc[:10]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri 2 wkli comp win fa cup final tkt 21st may 2005 text fa 87121 receiv entri question std txt rate c appli 08452810075over18',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl 3 week word back like fun still tb ok xxx std chg send ã â ã â 1 50 rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press 9 copi friend callertun',
 'winner valu network custom select receivea ã â ã â 900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobil 11 month u r entitl updat latest colour mobil camera free call mobil updat co free 08002986030']

In [26]:
cv = CountVectorizer()

In [28]:
# create my vocab
vc = cv.fit_transform(stemmed_doc)

In [29]:
X = vc.todense()

In [30]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [31]:
# NB from sklearn
from sklearn.naive_bayes import MultinomialNB

In [71]:
model = MultinomialNB()
model.fit(X_train, y_train)
model.score(X_test, y_test)*100

97.77052746057639

In [72]:
messages = [
    """
CLAIM BONUS

₹ 10,000 Welcome Package

+ 30 Free Spins

Casino Masters is the brand new casino from the people behind Genesis, Casino Gods and Casino Joy!       
Discover Casino Masters today and claim your welcome package of 100% up to ₹ 10,000 + 30 Free Spins when you make your first deposit.
You will find the same great games as all of the Genesis predecessor's plus all the brand new games added every month.
Click the button below to be one of the first to claim your Casino Masters bonus..
""",

"""
Hey,
Does the world of FINANCE fascinate you? Do you see yourself making a name out of yourself in the financial world?
Well, here is a perfect opportunity for you. 
David Eccles School of Business (DESB) at University of Utah is organising a webinar on details regarding their Masters course in Finance. 
The course, which is STEM-designed, offers a perfect opportunity for an amazing career in the finance industry! 
David Eccles School MSF is #ranked No. 1 in the U.S. for best value by the Financial Times, and top-13 overall by the TFE Times.
The course, which has a 95% employment rate,  helps you be prepared for changing dynamics of the financial world. 
Here's what you learn from the session: 

Course details
Application Process
Eligibility Criteria
Scholarships
Job Prospects
You also get waivers on GRE/GMAT & application fee by attending the session! 

So what are you waiting for?
"""
]

In [73]:
def prepare(messages):
    d = getDoc(messages)
    return cv.transform(d)

messages = prepare(messages)

In [77]:
y_pred = model.predict(messages)
y_pred

array(['spam', 'ham'], dtype='<U4')